In [1]:
from pandas import *

ImportError: Unable to import required dependencies:
numpy: 

IMPORTANT: PLEASE READ THIS FOR ADVICE ON HOW TO SOLVE THIS ISSUE!

Importing the numpy C-extensions failed. This error can happen for
many reasons, often due to issues with your setup or how NumPy was
installed.

We have compiled some common reasons and troubleshooting tips at:

    https://numpy.org/devdocs/user/troubleshooting-importerror.html

Please note and check the following:

  * The Python version is: Python3.10 from "/usr/local/bin/python3"
  * The NumPy version is: "1.22.2"

and make sure that they are the versions you expect.
Please carefully study the documentation linked above for further help.

Original error was: dlopen(/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/numpy/core/_multiarray_umath.cpython-310-darwin.so, 0x0002): tried: '/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/numpy/core/_multiarray_umath.cpython-310-darwin.so' (mach-o file

# Structure

0. Introduction
1. Data: characteristics, possible issues and resolutions
2. Software organization: where is it possible to find what?
3. Functioning
4. Specification and things that do not work

# Introduction

<p style="font-size: 1.2em; line-height: 1.5em">Our goal is to build a software that enables a possible user to query a relational and a graph database.</br>
The aim of the project is to create a software that can handle the processing and the storing of data coming from CSV and JSON files into different databases, as well as providing the possibility to query these databases simultaneously by having in return specific lists of Python objects.</br>
Firstly, we sterted by analysing the data we have been provided with in order to understand the different cases we had to handle and their characteristics. Then, the components used for the upload have been developed, taking in consideration the possibility of having to deal with only partial data. Here, some other issues contained into the starting data have been identified. As a consequence, we developed some additional functions aimed at correcting these problems. Finally, the query processors have been developed. Here we focused on the possibilty of creating a reliable piece of software that could be able to connect with different databases using different query languages, and to merge the information coming from the different origins in order to create complete python objects with respect to the classes defined with respect to our data model.</p>

_____________________________________________________

# 1. Data characteristics, possible issues and resolutions

<p style="font-size: 1.2em; line-height: 1.5em;">We started by analyzing the exemplar JSON and CSV files that we have been provided with to test the software.</br>
The CSV files are composed by the following columns:</p>

<code><strong>id, title, type, publication_year, issue, volume, chapter, publication_venue, venue_type, publisher, event</strong></code>

<p style="font-size: 1.2em; line-height: 1.5em;">Each row defines a publication entity and the column "publication_venue" the relationship of that publication with its venue by mean of the latter's title.</br>
When this column contains a value we know that the publications is contained in a venue, whose identifiers are specified in the JSON files along with additional data. In particular the JSON files are structured in 4 main keys:</p>

<code><strong> authors, venues_id, references, publishers </strong></code>

<p style="font-size: 1.2em; line-height: 1.5em;">The first three sections contain additional information about authors, venues and citations which are connected to the related publication by means of the publications' unique identifiers used as sub-keys inside each of these 4 macro "dictionaries". The fourth key give further information about the publishers that can be connected to the inforamtion of our csv through the mediation of their crossref identifier, which is used as key inside the json.</p>

<p style="font-size: 1.2em; line-height: 1.5em;">The files have been analysed both manually and by means of Python in order to better understand both the quantitative and the qualitative characteristics fo the data. Here we started facing some "problems" that have been considered during the development of our software.</p>

<p style="font-size: 1.2em; line-height: 1.5em;">The following cells show the quantitative characteristics of the data:</p>

In [ ]:
graph_publication = read_csv("data/graph_publications.csv")
graph_publication.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 503 entries, 0 to 502
Data columns (total 11 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   id                 503 non-null    object 
 1   title              503 non-null    object 
 2   type               503 non-null    object 
 3   publication_year   503 non-null    int64  
 4   issue              305 non-null    object 
 5   volume             393 non-null    object 
 6   chapter            93 non-null     float64
 7   publication_venue  488 non-null    object 
 8   venue_type         488 non-null    object 
 9   publisher          488 non-null    object 
 10  event              1 non-null      object 
dtypes: float64(1), int64(1), object(9)
memory usage: 43.4+ KB


In [ ]:
graph_publication.describe(include="all")

,id,title,type,publication_year,issue,volume,chapter,publication_venue,venue_type,publisher,event
count,503,503,503,503.000000,305,393,93.000000,488,488,488,1
unique,503,500,3,NaN,38,116,NaN,300,3,38,1
top,doi:10.1626637762/ueueeeeeeeeeeeeee,Supply Chain Coordination Based On Web Service,journal-article,NaN,1,11,NaN,Applied Sciences,journal,crossref:297,eruzione del Vesuvio
freq,1,2,409,NaN,63,22,NaN,15,408,174,1
mean,NaN,NaN,NaN,2019.127237,NaN,NaN,0.849462,NaN,NaN,NaN,NaN
std,NaN,NaN,NaN,1.780738,NaN,NaN,0.359536,NaN,NaN,NaN,NaN
min,NaN,NaN,NaN,2011.000000,NaN,NaN,0.000000,NaN,NaN,NaN,NaN
25%,NaN,NaN,NaN,2018.000000,NaN,NaN,1.000000,NaN,NaN,NaN,NaN
50%,NaN,NaN,NaN,2019.000000,NaN,NaN,1.000000,NaN,NaN,NaN,NaN
75%,NaN,NaN,NaN,2021.000000,NaN,NaN,1.000000,NaN,NaN,NaN,NaN


In [ ]:
relational_publication = read_csv("data/relational_publications.csv")
relational_publication.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 11 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   id                 500 non-null    object 
 1   title              500 non-null    object 
 2   type               500 non-null    object 
 3   publication_year   500 non-null    int64  
 4   issue              347 non-null    object 
 5   volume             443 non-null    object 
 6   chapter            22 non-null     float64
 7   publication_venue  498 non-null    object 
 8   venue_type         498 non-null    object 
 9   publisher          498 non-null    object 
 10  event              0 non-null      float64
dtypes: float64(2), int64(1), object(8)
memory usage: 43.1+ KB


In [ ]:
relational_publication.describe(include="all")

,id,title,type,publication_year,issue,volume,chapter,publication_venue,venue_type,publisher,event
count,500,500,500,500.000000,347,443,22.000000,498,498,498,0.0
unique,500,500,2,NaN,31,149,NaN,295,2,32,NaN
top,doi:10.1162/qss_a_00023,"Opencitations, An Infrastructure Organization ...",journal-article,NaN,1,11,NaN,Scientometrics,journal,crossref:297,NaN
freq,1,1,478,NaN,69,29,NaN,50,478,145,NaN
mean,NaN,NaN,NaN,2019.558000,NaN,NaN,0.909091,NaN,NaN,NaN,NaN
std,NaN,NaN,NaN,1.512353,NaN,NaN,0.294245,NaN,NaN,NaN,NaN
min,NaN,NaN,NaN,2014.000000,NaN,NaN,0.000000,NaN,NaN,NaN,NaN
25%,NaN,NaN,NaN,2018.000000,NaN,NaN,1.000000,NaN,NaN,NaN,NaN
50%,NaN,NaN,NaN,2020.000000,NaN,NaN,1.000000,NaN,NaN,NaN,NaN
75%,NaN,NaN,NaN,2021.000000,NaN,NaN,1.000000,NaN,NaN,NaN,NaN


In [ ]:
relational_other_data = read_json("data/relational_other_data.json")
relational_other_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 540 entries, doi:10.1162/qss_a_00023 to crossref:301
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   authors     508 non-null    object
 1   venues_id   498 non-null    object
 2   references  500 non-null    object
 3   publishers  32 non-null     object
dtypes: object(4)
memory usage: 21.1+ KB


In [ ]:
relational_other_data.describe(include="all")

,authors,venues_id,references,publishers
count,508,498,500,32
unique,486,297,99,32
top,"[{'family': 'Leydesdorff', 'given': 'Loet', 'o...","[issn:0138-9130, issn:1588-2861]",[],"{'id': 'crossref:6228', 'name': 'Codon Publica..."
freq,4,50,366,1


In [ ]:
graph_other_data = read_json("./data/graph_other_data.json")
graph_other_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 568 entries, doi:dioMadonna to crossref:4443
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   authors     530 non-null    object
 1   venues_id   490 non-null    object
 2   references  501 non-null    object
 3   publishers  38 non-null     object
dtypes: object(4)
memory usage: 22.2+ KB


In [ ]:
graph_other_data.describe(include="all")

,authors,venues_id,references,publishers
count,530,490,501,38
unique,496,313,101,38
top,"[{'family': 'Pal', 'given': 'Kamalendu', 'orci...",[issn:2076-3417],[],"{'id': 'crossref:9011001109', 'name': 'Nacc'a ..."
freq,4,15,332,1


## Problems

<p style="font-size: 1.2em; line-height: 1.5em;">The main things we have noticed by analysing the quantitaive aspects were:</p>
<ol style="font-size: 1.2em; line-height: 1.5em;">
    <li>The number of unique publications in the JSON and CSV files could be different, so even if they match, they do it only partially.</li>
    <li>The venues ids are multivalues and they can be defined both with a single or with a multiple id inside the JSON files.</li>
</ol>
<p style="font-size: 1.2em; line-height: 1.5em;">Other issues came from the structure of the files. In particular, in the CSV files defining the publication's entities, there are data relative to the venues, for ex. the title of the venues, that cannot be connected to a venue without an internal identifier, the only reliable source to identify and define unique venues. Information about the identifiers of the venues are only present in the JSON files, so those information cannot be added and the internal id of the venues cannot be created until a JSON that matched the CSV data is uploaded.</p>
<p style="font-size: 1.2em; line-height: 1.5em;">From the qualitative analysis, carried out by looking directly at the data contained in the exemplar files, we noticed that there are data related to the same identifiers that are reported in different ways. For example:</p>
<ol style="font-size: 1.2em; line-height: 1.5em;">
    <li>The given name of a person identified with the same identifier</li>
    <li>The same venue, connected with different publications, has always the same title but in some occurrences the "&" character is written as the XML entity reference "&Amp;".</li>
</ol>

## Solutions

<p style="font-size: 1.2em; line-height: 1.5em;">The main solutions we have identified in order to manage this issues are:</p>
<ol style="font-size: 1.2em; line-height: 1.5em;">
    <li>An external JSON file is automatically created by the execution of the upload of the data. Here the data of the venues are stored while waiting to be connected with their specific identifier. In particular, we developed a small function to name the file automatically depending on the endpoint or the path of the database. In this way each database that will be created, will have its own specifc JSON file from which data can be imported or even queried.</li>
    <li>When defining the venues by means of their ids, we have used a double iteration that checks first whether one of the ids is already assigned to a venue, and then iterate over the ids another time to bound them to the right internal id.</li>
    <li>A specific function handle the XML entities contained in the CSV files by replacing the most common entities with their respective charater each time the software reads a CSV file.</li>
    <li>Every time an already existing entity is encoutered, we keep the last version of the data, so if the user made a typing error or wants to update its data can easily do that by uploading another file. In the triplestore data processor this process is made by checking if the data related to it are the same and, if there is a new data, depending on the arity defined in the data model, we decide whether to add a new value or to keep the last added, as if an update occurred. Naturally we do that only for "static" data, so that the relations between different entities are not touched.</li>
</ol>

<p style="font-size: 1.2em; line-height: 1.5em;">Moreover, since the data types are interpreted every time they are uploaded to and downloaded from the databases, once in Python, we decided to convert them into string in order to have more possiblities for the manipulation. Accordingly to the specifications of the various predicates, they are converted into the right data type before the upload.
Naturally, once the databases are queried via generic query processor, the python object returned to the user contain data compliant with the data model.</p>


_________________________________________

# 2. Software organisation

<p style="font-size: 1.2em; line-height: 1.5em;">We organized the software dividing it in different pieces of code contained in different files that can be easily managed. In order to describe the aim and how each piece of software was developed we will devide the following section in subsections that have the same division and titles of the different files. </p>

### 2.1 classesDataModel

<p style="font-size: 1.2em; line-height: 1.5em;">The file contains the definition of the classes of the main data model, so the ones related to the entities that have to be returned to the user as Python object.</br>
It is imported in all the other files that make use of the classes here defined.</p><p>
<p style="font-size: 1.2em; line-height: 1.5em;">Each class has also its specific methods according to the data model. However, we decided to add a new method called <code>addCitedPublication()</code> to the class <code>Publication</code>, that will add each citation to the attribute cites. This has been done in order to avoid that the code could run in an infinite loop, which happens by adding the citations in the generic processor by means of the parameter of that class.


<div>
<img src="datamodel.png" width=80%/>
</div>

### 2.2 additionalClasses

<p style="font-size: 1.2em; line-height: 1.5em;">It contains the definition of the classes: <code>RelationalProcessor</code>, <code>QueryProcessor</code> and <code>TripleStoreProcessor</code>. We decided to put these three classes here because they are direct subclasses of the class <code>object</code>.</br>
This file is naturally imported in all the other files that make use of the classes here defined.</P>

<div>
<img src="images/uml2.png" width="800"/>
</div>

### 2.3 URIs

<p style="font-size: 1.2em; line-height: 1.5em;">
This file contains only the variables to which the URIs used by the triplestoreDatProcessor are assigned, and it is imported by every other file that uses them. It includes URIs that refer both to classes and properties.
</p>

### 2.4 relationalDataProcessor & triplestoreDataProcessor

<p style="font-size: 1.2em; line-height: 1.5em;">
In the two files, the classes <code>relationalDataProcessor</code> and <code>triplestoreDataProcessor</code> are defined.
They both have a simple structure that checks what kind of files have been already uploaded into the databases and, depending on the specific case, recalls an external function aimed at managing the upload of either a CSV or a JSON file.</br>
These functions, as well as the other functions used by all the processors are explained in another section (see 2.7 relationalUploadFunction & triplestoreFunctions).
</p>

<p style="font-size: 1.2em; line-height: 1.5em;">
We use the following functions in both the data processors in order to retrieve information from the csv and the json file.</p>

In [ ]:
from pandas import read_csv

path = "" #put there the path of a csv to see how it works

#READ THE CSV FILE
publications = read_csv(path,
keep_default_na=False,
dtype= {
    "id": "string",               
    "title" : "string",            
    "type" : "string" ,             
    "publication_year" : "string",   
    "issue" : "string",              
    "volume" :"string",              
    "chapter" :"string" ,         
    "publication_venue" : "string" ,  
    "venue_type" :"string",           
    "publisher" :"string" ,          
    "event" :"string"   
})

publications

FileNotFoundError: [Errno 2] No such file or directory: ''

In [ ]:
from json import load

nameOfJson = "" #put there the path of a csv to see how it works

#READ THE JSON FILE 
with open(nameOfJson, "r", encoding="utf-8") as jsonVenue:
    venueData = load(jsonVenue)

### 2.5 relationalQueryProcessor & triplestoreQueryProcessor

<p style="font-size: 1.2em; line-height: 1.5em;">
The files contain, respectively, the definition of the classes <code>relationalQueryProcessor</code> and <code>triplestoreQueryProcessor</code>.</br>
In the <code>relationalQueryProcessor</code>, the methods execute SQL queries by using the python library <code>sqlite3</code> with the aim of retrieving the desired data from the relational database. Here, when necessary, data are also retrieved from the external JSON containing the data related to the venues with missing data, i.e. their ids. Each method of the class returns a pandas DataFrame with the information necessary in order to build the python objects requested.</br>
In the <code>triplestoreQueryProcessor</code>, the methods execute SPARQL queries in order to return the desired data, while the handling of the pandas DataFrame, as well as the management of the data coming from the external JSON, are demanded to specific functions developed for these purposes.</p>
<p style="font-size: 1.2em; line-height: 1.5em;">
Some additional methods have been defined for both classes to handle the additional requests coming from the generic query processor when the retrieved data are not enough to create the complete Python object that has to be returned to the user. The methods are:
</p>
<ol style="font-size: 1.2em; line-height: 1.5em;">
    <li><code>getPublicationByDoi()</code>: it returns a dataframe with the information about a specific publication that has as identifier the input doi.</li>
    <li><code>getAuthorById()</code>: it returns a dataframe with the information about all the authors of a specific publicatioin that has as identifier the input doi.</li>
    <li><code>getPublisherById()</code>: it returns a dataframe with the information about the publisher of a publication that has as identifier the input doi.</li>
    <li><code>getVenuesByDoi()</code>: it returns a dataframe with the all the ids of a venue that contains the publication that has as identifier the input doi.</li>
</ol>

### 2.6 genericQueryProcessor

<p style="font-size: 1.2em; line-height: 1.5em;">
The class <code>genericQueryProcessor</code> contains only the methods shown in the data model, which call the corresponding methods of the relational and the triplestore query processors. These methods can call also some external functions that we defined with the purpose of addressing recurring cases that could be afford with the same code. Each method returns a list of python model-compliant object that are delivered to the user, ready to be resused with their own methods, defined in the file "classesDataModel" (see 2.1).
</p>

### 2.7 relationalUploadFunctions & triplestoreFunction

<p style="font-size: 1.2em; line-height: 1.5em;">
In the <code>relationalUploadFunctions</code>, can be found the functions called by the <code>relationalDataProcessor</code>.
In particular:
</p>
<ol style="font-size: 1.2em; line-height: 1.5em;">
    <li><code>replace_entity_reference()</code> is called to replace the occurrences of the most common entity references in the "title" and "publication_venue" column when reading a CSV file with pandas. It takes in input the dataframe coming from the CSV file and return the same dataframe with the correct replacements.</li>
    <li><code>upload_csv()</code> is called in the case in which a CSV has to be uploaded in the relational database. It takes in input the CSV file and the database's path and manages both the first upload and the update of the tables into the database.</li>
    <li><code>upload_csv()</code> is called when a JSON has to be uploaded in the relational database. It takes in input both the JSON file and the database's path and manages both the first upload and the update of the tables into the database.</li>
</ol>
<p style="font-size: 1.2em; line-height: 1.5em;">
In the second file, more functions used by the <code>triplestoreDataProcessor</code> and the <code>triplestoreQueryProcessor</code>, as well as some queries that are reused by the function themselves, are defined.
Aside from the queries, the functions are:
<ol style="font-size: 1.2em; line-height: 1.5em;">
    <li><code>csv_exists()</code> and <code>json_exists()</code> take in input the Blazegraph database's endpoint and return a boolean value. They are used by the data processor to check whether a CSV or a JSON file has been already uploaded into the database.</li>
    <li><code>replace_entity_reference()</code> is called to replace the occurrences of the most common entity references in the "title" and "publication_venue" column when reading a CSV file with pandas in the data processor. It takes in input the dataframe coming from the CSV file and return the same dataframe with the correct replacements.</li>
    <li><code>fix_df_columns_encoding()</code> is called by the triplestore query processor to fix the encoding of the literals coming from the Blazegraph database in order to return a data frame with the values encoded in utf-8 to the generic query processor. It takes in input a dataframe and three boolean values that are used to understand whether the dataframe contains columns related to publications, to venues and to events, and returns the same dataframe with the correct encoding.</li>
    <li><code>replace_dot_zeros()</code> is called by the triplestore query processor to fix the ".0" that appear in some columns when Python interpret the data types in the wrong way. It takes in input a dataframe and a boolean value used to understand when it has to deal with the column related to "chapter number", and returns the same dataframe with the correct replacements.</li>
    <li><code>upload_in_store()</code> is called by the data processor to upload the triples contained into the local graph to the Blazegraph database. It takes in input the instance of the class "SPARQLUpdateStore", the endpoint of the Blazegraph database, and the local graph containing the triples to upload.</li>
    <li><code>update_store()</code> is called by the data processor to both upload the triples contained into the local graph to the Blazegraph database and to delete the ones that contain data that have found to be up to be dated. It takes in input the same parameters as the previous function plus the other local graph containing the triples to delete.</li>
    <li><code>read_csv_file()</code> <code>read_json_file()</code> are just recalled when it is necessary to read one CSV or JSON file in pandas. They take in input the path of the file and return a pandas dataframe (the first) and a dictionary (the second).</li>
    <li><code>name_additional_data_file()</code> is called by the data processor to name the file and the path of the external JSON file depending on the endpoint URL of the database. It takes in input the Blazegraph endpoint and creates a new JSON file.</li>
    <li><code>upload_additional_data()</code> is called by the data processor to store the data related to the non-defined venues into the exteral JSON file. It takes in input the path of the JSON file that stores the additional data related to the venues and the dictionary containing the data to insert in the JSON itself.</li>
    <li><code>additional_data_dataframe()</code> is called by the triplestore query processor to create a dataframe from the JSON containing the information about the venues. This dataframe is created to be concatenated to the one built from the query ran by the called method. It takes in input the list containing the headers of the dataframe generated by the query, the path of the JSON from which the new dataframe has to be created, the list cof internal ids created from one of the columns of the query dataframe (depending on the query), the blazegraph endpoint, the key of the dictionary used to make the checks useful to create the dataframe from the JSON (depending on the query), the list of publisher internal ids or an empty string (depending on the query), the crossref identifier the user is searching for or an empty string (depending on the query), and the partial string the user used to retrieve a venue by its event or an empty string (depending on the query).</li>
    <li><code>retrieve_doi()</code> and <code>retrieve_crossref()</code> are two recursive functions called inside the previous function and are used to convert the internal identifiers of the publications and the publishers into doi and crossref identifiers. They take in input the Blazegraph endpoint, the list of internal ids that have to be converted and an empty list, and return the list of identifiers that will be used to create the specific column of the dataframe in the previous function.</li>
    <li><code>csv_upload()</code> and <code>json_upload()</code> are called by the data processor to upload the data contained, respectively, into a CSV file or a JSON file to the Blazegraph database when the latter is empty.</li>
    <li><code>csv_to_csv()</code> and <code>csv_to_json()</code> are called by the data processor to upload the data contained into a CSV file to the Blazegraph database when it contains, respectively, either data coming from the CSV or data coming from a JSON file.</li>
    <li><code>json_to_csv()</code> and <code>json_to_json()</code> are called by the data processor to upload the data contained into a JSON file to the Blazegraph database when it contains, respectively, either data coming from the CSV or data coming from a JSON file.</li> is called by the data processor to name of the file and the path of the external JSON file depending on the endpoint URL of the database.</li>    
</ol>
</p>


_________________________________________________________

## 3. Functioning

### 3.1 Relational Processor

<p style="font-size: 1.2em; line-height: 1.5em;">
It is the parent class of both the <code>RelationalDataProcessor()</code> and the <code>RelationalQueryProcessor</code> and has the attribute <code>dbPath</code> used to get and change the path of the database through two functions: 
<ul>
<li><code>getDbPath()</code>: it returns the path of the database.</li>
<li><code>setDbPath()</code>: it allows to changes the path of the database.</li>
</ul>
</p>

### 3.2 Relational Data Processor - Upload Data

<p style="font-size: 1.2em; line-height: 1.5em;">
The class <code>RelationalDataProcessor()</code> has only the method <code>uploadData()</code> that takes in input the path of the file the user wants to update that can be both a csv or a json. The function checks if the file is empty, otherwise it checks weather it has the <code>.csv</code> extension or the <code>.json</code> extension. In the first case it will run the external function <code>upload_csv()</code>, otherwise it will run the external function <code>upload_json()</code>.
</p> 

#### 3.2.1 <code>upload_csv()</code>

<p style="font-size: 1.2em; line-height: 1.5em;">
With information from a CSV we are able to populate the tables JournalArticles, BookChapters, PorceedingsPapers, PublicationsIds, PublishersIds. </br>
We also have information about the venue name and the venue type but, since we have no information about the venue identifier, we can't complete our venues' tables yet. As a matter of fact, we can't be sure that two venues with the same name are actually the same venue, we will only be able to uniquely identify them, when receiving identifiers information from a JSON.
</br>
However, for practicality and in order to handle better SQL queries, that raise error when asking for a table that is not in the database, in the class <code>RelationalQueryProcessor</code> of our software, we opted for the creation of <u>all the tables with all the headers, leaving empty values where there are no information for to add yet</u>. These data will be added later when a JSON is uploaded.
</p>
<p style="font-size: 1.2em; line-height: 1.5em;">
Those information will not be updated in the following cases:
<ol style="font-size: 1.2em; line-height: 1.5em;">
<li>if only a CSV is passed as input of the uploadData function with the current database path.</li>
<li>Both CSV and JSON files are passed as input in the current databse but their information do not match with each other (ex. the publication identifiers in the JSON are different from those present in the CSV)</li>
<li>The arrity of the attribute/relation can be 0, meaning that the information could not be present, but the entity is still compliant with our data model.</li>
</ol>
</p>

<p style="font-size: 1.2em; line-height: 1.5em;">
In order to know if the tables must be uploaded for the first time or wether they are already in the database we use some try/except blocks: in the try block we ask for the tables, if there aren't the code will procede with the except block that will create those tables. 
</p>

<p style="font-size: 1.2em; line-height: 1.5em;">
The code is structured as follows: 
<ul style="font-size: 1.2em; line-height: 1.5em;">
<li>First we read the CSV file and we assign the type string to all the columns</li> 
<li>The first try/except code, we check whether the tables that can be created both from a JSON file and a CSV file, i.e. PublicationsIds and PublishersIds, are already in the database. If there are not, we create the tables for the first time.</li>
<li>Outside of the try - except, we create the tables of the publications by querying the whole dataframe created from the csv, asking for rows of a specific type of publication and merging the new dataframe with the PublicationsIds in order to add the internal id and we keep for each table, i.e. JournalArticles, BookChapters and ProceedingsPapers, only the columns that are compliant to the data model. 
<li>In another try/except clause we update the publications' tables if they already exists we the new data and we add (or update in case it is not the first file).
</li>
<li>In the same try/except clause we check wether there are already the venues' tables in order to update them with the new data. The data about the venues that do not match a venue id will be loaded in an external JSON (see 2.5). If the tables of the veneus are not already in the database all the data will be directly loaded in the external JSON</li>
<li>If none of those tables was already in the database the one that can be created only by means of a JSON file, e.g. AuthorsIds, will be created as empty dataframe with a header compliant to the data model.</li>
<li>Finally we upload all the tables in the database.</li>
</ul>


#### 3.2.2 <code>upload_json()</code>

<p style="font-size: 1.2em; line-height: 1.5em;">
For the <code>upload_json()</code> function, the overall logic we followed is the same:
<ul style="font-size: 1.2em; line-height: 1.5em;">
<li>First we see if the PublicationsIds and the PublishersIds must be updated or it is the first time we upload them. We ask also the VenuesIds table and the Publishers tables, since we can immediately create them and we know that all the tables are created with the first upload.</li>
<li>If it is the first file uploaded in the database we are able to create and populate the following tables: PublicationsIds, Publishers, PublishersIds, VenuesIds, PublicationsVenues, Citations, Authors, AuthorsIds. All the other tables will be created as empty dataframes with headers compliant to the data model.</li>
<li>If those tables are already present in the database, i.e. the SQL's queries succeed, all the tables mentioned above are updated.</li>
<li>If there are already tables inside the database, through PublicationsVenues table, we associate the venue internal id as a foreign key to ours venues' tables, i.e. Journals, Books and Proceedings. We also download the venues information coming from the CSV that were previously stored in our additional JSON file, and see if they can be associated through the doi to the venues' tables.</li>
</ul>
</p>

### 3.3 Triplestore Processor

<p style="font-size: 1.2em; line-height: 1.5em;">
It is the parent class of both the <code>TriplestoreDataProcessor()</code> and the <code>TriplestoreQueryProcessor</code> and has the attribute <code>endPointUrl</code> used to get and change the enpoint of the database through two functions: 
<ul>
<li><code>getEndPointUrl()</code>: it returns the path of the database.</li>
<li><code>setEndPointUrl()</code>: it allows to changes the path of the database.</li>
</ul>
</p>

### 3.4 Triplestore Data Processor

<p style="font-size: 1.2em; line-height: 1.5em;">
The <code>uploadData()</code> method of the <code>TriplestoreDataProcessor()</code> has the same aim of the one of the <code>RelationalDataProcessor()</code>. 
The function checks at first the existence of a string type path. If this basic condition is fulfilled, the methods go on with additional checks on the kind of path that is defined as input.</br>
These additional checks are in particular two: the first one is useful for CSV formats, while the second one for JSON formats. Seen the fact that the upload is thought only for JSON and CSV, if the path of the file is not of one of these two different types the upload method fails, returning False to the user.
</br>
Once inside one of these two conditions, the methods checks if inside our database we have yet data coming from a CSV, a JSON, or coming by both. According to the file path and to the previous uploads, i.e. to the fact that there can be already data coming from a JSON or from a CSV file, it executes of the six functions according to the scenarios explained below:
</p>

<ol style="font-size: 1.2em; line-height: 1.5em;">
<li><code>csv_upload()</code>: it uploads data coming from a CSV file when the database is empty</li>
<li><code>json_upload()</code>: it uploads data coming from a JSON file when the database is empty</li>
<li><code>csv_to_csv()</code>: it uploads data coming from a CSV when another CSV was already uploaded in the database</li>
<li><code>json_to_csv()</code>: it uploads data coming from a JSON when a CSV was already uploaded in the database</li>
<li><code>json_to_json()</code>: it uploads data coming from a JSON when another JSON was already uploaded in the database</li>
<li><code>csv_to_json()</code>: it uploads data coming from a CSV when a JSON was already uploaded in the database</li>
</ol>

### 3.5 Query Processor

<p style="font-size: 1.2em; line-height: 1.5em;">
It is the parent class of both the <code>RelationalQueryProcessor()</code> and the <code>TriplestoreQueryProcessor()</code> and it has no attributes or relations.
</p>

### 3.6 Relational Query Processor

<p style="font-size: 1.2em; line-height: 1.5em;">
The structure of the <code>RelationalQueryProcessor()</code> has been defined above (see 2.5). 
We managed the queries by means of the library <code>sqlite3</code>. The main syntax of SQL that we used is the follows: 
<ul style="font-size: 1.2em; line-height: 1.5em;">
<li>SELECT <\name_of_the_column>: to select the columns we wanted in our dataframe.</li>
<li>LEFT JOIN <\table1> ON <\table2>.<\name_of_the_column> == <\table1>.<\name_of_the_column>: to join the tables on the columns that have the same values</li>
<li>UNION ALL: to join the results of queries</li>
<li>WHERE <\name_of_the_column> = '"""+<\name_of_the_python_variable>+"""': to set the condition according to the variable in input in the queries</li>
</ul>
</p>
<p style="font-size: 1.2em; line-height: 1.5em;">
We used also an external function <code>getVenues()</code> in order to retrieve a dataframe with all the data of the venues that still did not match any venue id, i.e. that have not be added to the database since they are not compliant to the data model, but that still can have useful information for our query. The functions takes in input the dataframe of the method that calls it in order to return a dataframe with the same headers that can be easily add to the output dataframe.
</p>

### 3.7 Triplestore Query Processor

<p style="font-size: 1.2em; line-height: 1.5em;">
The structure of the <code>RelationalQueryProcessor()</code> has been defined above (see 2.5). 
We managed the queries by means of the library <code>sparql_dataframe</code>.
<ul style="font-size: 1.2em; line-height: 1.5em;">>
<li>SELECT ?variable: it sets only the values that must be returned by the query</li>
<li>WHERE {<\condition>}: inside the brackets there is a triple that is the condition that must be fullfilled</li>
<li>OPTIONAL {<\condition>}: inside the brackets there is a triple that is a condition that it is not necessary in order to return the values </li>
<li>FILTER: it filters the results according to the value of a variable</li>
<li>FILTER NOT EXIST {}: it filters the elements that do not have the triple inside the brackets</li>
<li>UNION: to join the results of the queries</li>
</ul>

### 3.8 Generic Query Processor

<p style="font-size: 1.2em; line-height: 1.5em;">
The structure and the aims of the class have already been described above (see 2.6).
It recalls some external functions that we added in order to run common patterns: 
<ul style="font-size: 1.2em; line-height: 1.5em;">
<li><code>getPublicationObject()</code>: it is a recursive function that takes in input a doi, a dictionary, a dataframe and the list of the query processors objects. It has the aim of create a publication object and it is recursive because it creates also the python object of the publications it cites (and of the publications cited by its citation until it matches a base case). It returns a tuple with a dictionary with all the objects created and the python object of the publication we wanted to create. If it is not possible to create the python object (this happens when necessary information are not complete) it returns the object as an empty string in order to return also this information to the function that calls it.</li>
<li><code>searchForCitationObject()</code>: it takes in input two dictionaries and the list of the query processors objects and it returns a dictionary of publications and a dictionary with the publications without type.</li>
<li><code>addPublicationToDict()</code>: it takes in input a row, a dictionary of publications and the dictionaries of publications without type and it returns the two updated dictionaries</li>
<li><code>updatePublicationWithType()</code>: it takes in input a row, a dictionary of publications and the dictionaries of publications without type and it returns the two dictionaries with the update of the publications that now have a type.</li>
<li></code>getMostCitedInDataframe()</code>: it is a recursive function that takes in input the dictionary of the counts, the dictionary of the publications objects and the list of the query processors objects and returns the object with the most number of citations.</li>
</ul>
</p>

________________________________

# 4. Specifications

<p style="font-size: 1.2em; line-height: 1.5em;">
The software returns only objects compliant with the database. It means that an object of class JpurnalArticle citing two other publications, will contain, in the attribute "cites" only compliant objects. If one of these publication objects cannot be created, the information about that specific citation will be lost.
In other words, if the sum of the data contained into the different databases is still incomplete in some way, the object that the software returns will be always compliant to the data model, therefore containing compliant but partial information. </p>